In [ ]:
!pip install pymorphy2 pyLDAvis==3.4.1 bigartm corus razdel numpy==1.25.1

In [55]:
import numpy as np
import random
import re
import nltk
import artm
import pyLDAvis
import razdel
import csv
import json
import gensim
import pyLDAvis.gensim_models as gensimvis

from bs4 import BeautifulSoup
from corus import load_lenta
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora

import warnings
warnings.filterwarnings("ignore")

In [8]:
RANDOM_STATE = 2023
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

**Загрузка данных и препроцессинг**

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")

In [10]:
sw = stopwords.words('russian')
additional_sw =  'мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы '
pronouns = 'я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий'
conjunctions = 'что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда'
digits = 'ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто'
modal_words = 'вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется'
particles = 'да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли'
prepositions = 'близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно'
evaluative = 'наиболее наименее лучший больший высший низший худший более менее'

sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

In [31]:
with open("datasets/lenta_data.json", "r") as file:
    dataset = json.load(file)

In [32]:
dataset[0]

'Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.'

In [33]:
def clean(row):
    clean = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    clean_text = re.sub(clean, '', row)
    return clean_text

def clean_with_bs4(content):
    soup = BeautifulSoup(content, 'html.parser')
    return soup.get_text()

In [ ]:
analyzer = nltk.WordNetLemmatizer()
lemmatized_sw = [analyzer.lemmatize(word) for word in sw]

def tokenizer(text, stopwords, need_lemmatize=False):
    result = []
    sentences = [item.text for item in razdel.sentenize(str(text))]

    for sentence in sentences:
        text = sentence.lower()
        text = clean_with_bs4(text)
        text = re.sub(r"\s+", ' ', text)

        tokens = [item.text for item in  razdel.tokenize(text)]
        tokens = [re.sub("[^а-яА-Яa-zA-Z]", ' ', item) for item in tokens]

        if need_lemmatize:
            tokens = [analyzer.lemmatize(token) for token in tokens if token not in stopwords  and ' ' not in token and len(token) > 2]
            tokens = [token for token in tokens if token not in lemmatized_sw]
        tokens = [re.sub(r"ё", "е", token) for token in tokens]
        result.extend(tokens)
    return result

tokenized = [tokenizer(text, stopwords=sw, need_lemmatize=True) for text in tqdm(dataset)]

In [35]:
tokenized_text = [' '.join(text) for text in tokenized]

**Обучпение BigARTM модели**

In [39]:
count_vectorizer = CountVectorizer(max_features=1000, max_df=0.8, min_df=2, ngram_range=(1, 2))
n_wd = np.array(count_vectorizer.fit_transform(tokenized_text).todense()).T

tokens_list = [i for i in count_vectorizer.vocabulary_.keys()]

bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=tokens_list)

In [41]:
def params(space):
    model = artm.ARTM(num_topics=space['num_topics'], dictionary=bv.dictionary, cache_theta=True)

    model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
            name='SparsePhi',
            tau=space['phi_tau']
        ),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=space['theta_tau']
        ),
    )
    model.regularizers.add(
        artm.DecorrelatorPhiRegularizer(
            name='DecorrelatorPhi',
            tau=space['decorrelation_tau']
        ),
    )

    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    print("Perplexity score: ", perplexity_score)
    return {'loss': perplexity_score, 'status': STATUS_OK }

def search_params():

    space={
        'num_topics': hp.choice('num_topics', topics_to_check),
        'phi_tau': hp.uniform('SparsePhi', -1, 1),
        'theta_tau': hp.uniform('SparseTheta', -1, 1),
        'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
    }
    trials = Trials()
    best_params = fmin(
        fn=params,
        space=space,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials,
    )
    return best_params

In [42]:
check_topics =  [2, 5, 15, 40, 100]

def search_params():

    space={
        'num_topics': hp.choice('num_topics', check_topics),
        'phi_tau': hp.uniform('SparsePhi', -1, 1),
        'theta_tau': hp.uniform('SparseTheta', -1, 1),
        'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
    }
    trials = Trials()
    best_params = fmin(
        fn=params,
        space=space,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials,
    )
    return best_params

In [ ]:
best_estimator = search_params()

In [44]:
best_estimator

{'DecorrelatorPhi': 17547.471889933066,
 'SparsePhi': -0.249538620314941,
 'SparseTheta': 0.09463238715518352,
 'num_topics': 3}

In [45]:
def fit_model(seed=RANDOM_STATE):
  model = artm.ARTM(num_topics=check_topics[best_estimator['num_topics']], dictionary=bv.dictionary, cache_theta=True, seed=seed)
  model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                        dictionary=bv.dictionary))

  model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

  model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=best_estimator['SparsePhi']))
  model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=best_estimator['SparseTheta']))
  model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=best_estimator['DecorrelatorPhi']))

  model.fit_offline(bv, num_collection_passes=30)

  return model

In [47]:
model = fit_model()
print(f'Perplexity score - {model.score_tracker["perplexity_score"].last_value}')

Perplexity score - 249.15711975097656


**pyLDAvis визуализация**

In [49]:
phi = model.get_phi()
theta = model.get_theta().to_numpy().T
theta = theta / theta.sum(axis=1, keepdims=1)
data_vis = {'topic_term_dists': phi.to_numpy().T,
        'doc_topic_dists': theta,
        'doc_lengths': n_wd.sum(axis=0).tolist(),
        'vocab': phi.T.columns,
        'term_frequency': n_wd.sum(axis=1).tolist()}


In [50]:
model_vis = pyLDAvis.prepare(**data_vis)

In [51]:
pyLDAvis.display(model_vis)

In [52]:
pyLDAvis.save_html(model_vis, 'bigartm_vis.html')

**Модель LDA**

In [57]:
bigram = gensim.models.Phrases(
    tokenized, min_count=5, threshold=100
)
trigram = gensim.models.Phrases(
    bigram[tokenized], threshold=100
)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [58]:
data_lda = [bigram_mod[doc] for doc in tokenized]
data_lda = [trigram_mod[bigram_mod[doc]] for doc in tokenized]

In [60]:
id2word = corpora.Dictionary(data_lda)

In [61]:
corpus = [id2word.doc2bow(text) for text in data_lda]

In [62]:
for n_topics in [5, 10, 15]:
    for iterations in [100, 150, 200]:
        for passes in [50, 100, 150]:
            lda_model = gensim.models.ldamulticore.LdaMulticore(
                corpus=corpus,
                id2word=id2word,
                workers=4,
                num_topics=n_topics,
                random_state=42,
                chunksize=10,
                passes=100,
                alpha="symmetric",
                iterations=150,
                eval_every=10,
                per_word_topics=True,
            )

In [63]:
model_lda = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    workers=4,
    num_topics=15,
    random_state=42,
    chunksize=10,
    passes=100,
    alpha="symmetric",
    iterations=200,
    eval_every=10,
    per_word_topics=True,
)

print(model_lda.print_topics())

[(0, '0.011*"мужчина" + 0.010*"женщина" + 0.010*"дома" + 0.010*"рассказала" + 0.007*"городе" + 0.007*"полиции" + 0.006*"люди" + 0.006*"результате" + 0.006*"дом" + 0.006*"мать"'), (1, '0.013*"день" + 0.012*"работы" + 0.009*"военные" + 0.006*"актер" + 0.006*"матч" + 0.006*"решили" + 0.006*"своим" + 0.005*"слишком" + 0.005*"сил" + 0.005*"работа"'), (2, '0.013*"фсб" + 0.008*"франции" + 0.008*"союза" + 0.007*"лукашенко" + 0.006*"подтвердил" + 0.005*"просьбой" + 0.005*"мкс" + 0.005*"украинского" + 0.005*"выступал" + 0.004*"водителей"'), (3, '0.010*"нефти" + 0.009*"конкурса" + 0.007*"санкций" + 0.007*"отказаться" + 0.006*"нефть" + 0.006*"составит" + 0.006*"крым" + 0.005*"помочь" + 0.005*"фонд" + 0.005*"сняли"'), (4, '0.032*"россии" + 0.022*"года" + 0.018*"сша" + 0.016*"заявил" + 0.016*"словам" + 0.014*"декабря" + 0.013*"ноября" + 0.012*"году" + 0.012*"страны" + 0.010*"ранее"'), (5, '0.011*"рост" + 0.009*"составил" + 0.008*"встрече" + 0.007*"google" + 0.007*"bloomberg" + 0.007*"продукты" + 0.0

In [65]:
model_lda_vis = gensimvis.prepare(model_lda, corpus, dictionary=model_lda.id2word)
pyLDAvis.display(model_lda_vis)

In [66]:
pyLDAvis.save_html(model_vis, 'LDA_vis.html')